In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import os
from glob import glob
from pathlib import Path
import camelot

In [2]:
pdf_path = "C:\\Users\\SDotson\\Downloads\\epa_scghg_report_draft_0.pdf"

In [5]:
tables = camelot.read_pdf(pdf_path, pages="87", flavor="stream")

In [18]:
df = tables[0].df.loc[4:].drop(labels=[5,6])

In [15]:
dcr_list = [2.5, 2, 1.5]
emit_list = ["CO2", "CH4", "N2O"]

In [17]:
pd.MultiIndex.from_product([emit_list, dcr_list])

MultiIndex([('CO2', 2.5),
            ('CO2', 2.0),
            ('CO2', 1.5),
            ('CH4', 2.5),
            ('CH4', 2.0),
            ('CH4', 1.5),
            ('N2O', 2.5),
            ('N2O', 2.0),
            ('N2O', 1.5)],
           )

In [24]:
df = df.set_index(0)

In [25]:
df.columns = pd.MultiIndex.from_product([emit_list, dcr_list])

In [34]:
df.reset_index().drop(labels=[0,1,2]).set_index(0)

CO2              CH4                   N2O                  
      2.5  2.0  1.5    2.5    2.0    1.5     2.5      2.0      1.5
0                                                                 
2020  120  190  340  1,300  1,600  2,300  35,000   54,000   87,000
2030  140  230  380  1,900  2,400  3,200  45,000   66,000  100,000
2040  170  270  430  2,700  3,300  4,200  55,000   79,000  120,000
2050  200  310  480  3,500  4,200  5,300  66,000   93,000  140,000
2060  230  350  530  4,300  5,100  6,300  76,000  110,000  150,000
2070  260  380  570  5,000  5,900  7,200  85,000  120,000  170,000
2080  280  410  600  5,800  6,800  8,200  95,000  130,000  180,000

In [ ]:
scc = pd.read_excel("data/scc_mult.xlsx")


In [3]:
scc

,eall,t,Value
0,CO2,2010,150
1,NaN,2015,170
2,NaN,2020,190
3,NaN,2023,202
4,NaN,2026,214
5,NaN,2029,226
6,NaN,2032,238
7,NaN,2038,262
8,NaN,2041,274
9,NaN,2044,286
